In [1]:
from nnsight import LanguageModel
import torch
import nnsight
from nnsight import NNsight
torch.set_default_device('cuda')



In [3]:
# pip install bitsandbytes accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_8bit=True)

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-9b")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-9b",
    quantization_config=quantization_config,
)

input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=32)
print(tokenizer.decode(outputs[0]))


`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

The 'batch_size' attribute of HybridCache is deprecated and will be removed in v4.49. Use the more precisely named 'self.max_batch_size' attribute instead.


<bos>Write me a poem about Machine Learning.

I’m not sure if I’m the right person to write a poem about Machine Learning. I’m not a poet, and I’m


In [110]:
def nt(func):
    # nnsight tuple wrapper
    def wrapper(x):
        if isinstance(x, tuple):
            t = x[0]
            assert isinstance(x[0], torch.Tensor), "nnsight only goes depth 1, please make sure all tuples contain tensors"
            t_out = func(t)
            return((t_out, *x[1:]))
        elif isinstance(x, torch.Tensor):
            return func(x)
    return wrapper


def tokenize(text):
    return tokenizer(text, return_tensors="pt").to("cuda")['input_ids']

In [109]:
wrapped_model = NNsight(model)

@nt
def add_noise(x):
    # print(x.shape) # torch.Size([1, 7, 3584])
    return x + 0.01 * torch.randn_like(x)

with wrapped_model.trace(tokenize("i like pie, it is")):
    l_output = wrapped_model.model.layers[20].output
    nnsight.apply(add_noise, l_output)

torch.Size([1, 7, 3584])
